# PEFT - Lora 

## Step1 導入相關套件

In [2]:
import torch

cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")


CUDA available: True


In [3]:
# datasets
# transformers
# peft
# accelerate

In [4]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加載數據集

In [5]:
ds = load_dataset("c-s-ale/alpaca-gpt4-data-zh", split="train[:50%]")
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 24409
})

In [6]:
ds[:3]

{'instruction': ['保持健康的三个提示。', '三原色是什么？', '描述原子的结构。'],
 'input': ['', '', ''],
 'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。',
  '原子是物质的基本单位，它由三种基本粒子组成：质子、中子和电子。质子和中子形成原子核，位于原子中心，核外的电子围绕着原子核运动。\n\n原子结构具有层次性。原子核中，质子带正电，中子不带电（中性）。原子核非常小且致密，占据了原子总质量的绝大部分。电子带负电，通常围绕核运动，形成若干层次，称为壳层或电子层。电子数量与质子数量相等，使原子呈电中性。\n\n电子在每个壳层中都呈规律分布，并且不同壳层所能容纳的电子数也不同。在最里面的壳层一般只能容纳2个电子，其次一层最多可容纳8个电子，再往外的壳层可容纳的电子数逐层递增。\n\n原子核主要受到两种相互作用力的影响：强力和电磁力。强力的作用范围非常小，主要限制在原子核内，具有极强的吸引作用，使核子（质子和中子）紧密结合在一起。电磁力的作用范围较大，主要通过核外的电子与原子核相互作用，发挥作用。\n\n这就是原子的基本结构。原子内部结构复杂多样，不同元素的原子核中质子、中子数量不同

## Step3 數據集預處理

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [9]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 24409
})

In [10]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 三原色是什么？\n\nAssistant: 三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。</s>'

In [11]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。</s>'

## Step4 創建模型

In [12]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")

In [13]:
for name, parameter in model.named_parameters():
    print(name)

transformer.word_embeddings.weight
transformer.word_embeddings_layernorm.weight
transformer.word_embeddings_layernorm.bias
transformer.h.0.input_layernorm.weight
transformer.h.0.input_layernorm.bias
transformer.h.0.self_attention.query_key_value.weight
transformer.h.0.self_attention.query_key_value.bias
transformer.h.0.self_attention.dense.weight
transformer.h.0.self_attention.dense.bias
transformer.h.0.post_attention_layernorm.weight
transformer.h.0.post_attention_layernorm.bias
transformer.h.0.mlp.dense_h_to_4h.weight
transformer.h.0.mlp.dense_h_to_4h.bias
transformer.h.0.mlp.dense_4h_to_h.weight
transformer.h.0.mlp.dense_4h_to_h.bias
transformer.h.1.input_layernorm.weight
transformer.h.1.input_layernorm.bias
transformer.h.1.self_attention.query_key_value.weight
transformer.h.1.self_attention.query_key_value.bias
transformer.h.1.self_attention.dense.weight
transformer.h.1.self_attention.dense.bias
transformer.h.1.post_attention_layernorm.weight
transformer.h.1.post_attention_layernor

## Lora

### PEFT Step1 訓練文件

In [14]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM, modules_to_save=["word_embeddings"])
config ##Liz is beauty and smart

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=['word_embeddings'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

### PEFT Step2 創建模型

In [15]:
model = get_peft_model(model, config)

In [16]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='Langboat/bloom-1b4-zh', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['query_key_value'], lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=['word_embeddings'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [17]:
for name, parameter in model.named_parameters():
    print(name)

base_model.model.transformer.word_embeddings.original_module.weight
base_model.model.transformer.word_embeddings.modules_to_save.default.weight
base_model.model.transformer.word_embeddings_layernorm.weight
base_model.model.transformer.word_embeddings_layernorm.bias
base_model.model.transformer.h.0.input_layernorm.weight
base_model.model.transformer.h.0.input_layernorm.bias
base_model.model.transformer.h.0.self_attention.query_key_value.weight
base_model.model.transformer.h.0.self_attention.query_key_value.bias
base_model.model.transformer.h.0.self_attention.query_key_value.lora_A.default.weight
base_model.model.transformer.h.0.self_attention.query_key_value.lora_B.default.weight
base_model.model.transformer.h.0.self_attention.dense.weight
base_model.model.transformer.h.0.self_attention.dense.bias
base_model.model.transformer.h.0.post_attention_layernorm.weight
base_model.model.transformer.h.0.post_attention_layernorm.bias
base_model.model.transformer.h.0.mlp.dense_h_to_4h.weight
base_m

In [18]:
model.print_trainable_parameters()

trainable params: 190,582,784 || all params: 1,399,189,504 || trainable%: 13.620941513294827


## Step5 配置訓練參數

In [19]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=16,
    logging_steps=20,
    save_steps=20,
    num_train_epochs=1
)

## Step6 創建訓練器

In [20]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)



## Step7 模型訓練

In [21]:
trainer.train()

  0%|          | 0/190 [00:00<?, ?it/s]

{'loss': 2.3365, 'grad_norm': 6.139095783233643, 'learning_rate': 4.473684210526316e-05, 'epoch': 0.1}
{'loss': 2.2105, 'grad_norm': 5.305870532989502, 'learning_rate': 3.9473684210526316e-05, 'epoch': 0.21}
{'loss': 2.1614, 'grad_norm': 5.588064193725586, 'learning_rate': 3.421052631578947e-05, 'epoch': 0.31}
{'loss': 2.133, 'grad_norm': 5.154468059539795, 'learning_rate': 2.8947368421052634e-05, 'epoch': 0.42}
{'loss': 2.0894, 'grad_norm': 5.204080581665039, 'learning_rate': 2.368421052631579e-05, 'epoch': 0.52}
{'loss': 2.0653, 'grad_norm': 5.318876266479492, 'learning_rate': 1.8421052631578947e-05, 'epoch': 0.63}
{'loss': 2.0532, 'grad_norm': 5.090636253356934, 'learning_rate': 1.3157894736842106e-05, 'epoch': 0.73}
{'loss': 2.0536, 'grad_norm': 5.141245365142822, 'learning_rate': 7.894736842105263e-06, 'epoch': 0.84}
{'loss': 2.0386, 'grad_norm': 4.722776889801025, 'learning_rate': 2.631578947368421e-06, 'epoch': 0.94}
{'train_runtime': 2470.3626, 'train_samples_per_second': 9.881

TrainOutput(global_step=190, training_loss=2.1236141104447213, metrics={'train_runtime': 2470.3626, 'train_samples_per_second': 9.881, 'train_steps_per_second': 0.077, 'train_loss': 2.1236141104447213, 'epoch': 1.0})

## Step8 模型推理

In [22]:
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 在各大考试中，有很多需要掌握的技巧和策略可以帮助我们更好的应对考试。以下是总结一些考试技巧：\n\n1. 自我检查：每到一个科目，我们都应该做好自我检查。首先，要检查自己的复习进度，确定自己是否还有多少时间、精力来完成试卷上的每一项作业;其次，可以检查自己对于每项作业的质量和准确性，如果有疏漏或错误，可以记录下来加以修正。\n\n2. 注意：对于考试科目有很多，我们无法面面俱到，所以在做试卷'